In [3]:
import time
import json
import traceback
import pathlib

import selenium
from selenium import webdriver
from selenium.common.exceptions import (
    NoSuchWindowException,
    TimeoutException,
)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.common.keys import Keys

# If you have these utility modules, adjust imports accordingly:
# from utils import dismiss_alert, bring_to_front
# from login_xiaohongshu import XiaoHongShuLogin

# ---------------------------------------------------------------------
# For demonstration, we'll define minimal placeholders for 
# `dismiss_alert`, `bring_to_front`, and `XiaoHongShuLogin` here.
def dismiss_alert(driver):
    try:
        alert = driver.switch_to.alert
        alert.accept()
        print("Alert was present and accepted.")
    except:
        pass

def bring_to_front(window_titles):
    # Stub: Implement your logic to focus on the specified window.
    print(f"Focusing on {window_titles} if found. (Stub)")

class XiaoHongShuLogin:
    # Stub: A placeholder if your login flow is already handled.
    def __init__(self, driver):
        pass
    def check_and_act(self):
        print("Checking if XHS is logged in. (Stub)")

# ---------------------------------------------------------------------
# ChromeDriver creation function
chromedriver_path = '/usr/lib/chromium-browser/chromedriver'

def create_new_driver(port=5006):
    """
    Creates a new Chrome WebDriver instance connected to an existing
    Chrome debugger at the given port, using the provided chromedriver_path.
    """
    options = webdriver.ChromeOptions()
    options.add_experimental_option("debuggerAddress", f"127.0.0.1:{port}")
    service = Service(executable_path=chromedriver_path)
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# ---------------------------------------------------------------------
class XiaoHongShuPublisher:
    def __init__(self, driver, path_mp4, path_cover, metadata, test=False):
        self.driver = driver
        self.path_mp4 = path_mp4
        self.path_cover = path_cover
        self.metadata = metadata
        self.test = test
        self.retry_count = 0  # initialize retry count

        xhs_login = XiaoHongShuLogin(driver)
        xhs_login.check_and_act()

    def wait_for_element_to_be_clickable(self, xpath, timeout=600):
        driver = self.driver
        try:
            WebDriverWait(driver, timeout).until(
                lambda d: d.find_element(By.XPATH, xpath).is_displayed()
                          and d.find_element(By.XPATH, xpath).is_enabled()
                          and "disabled" not in d.find_element(By.XPATH, xpath).get_attribute("class")
                          and d.find_element(By.XPATH, xpath).value_of_css_property("opacity") == "1"
            )
            print(f"Element {xpath} is truly interactable.")
        except TimeoutException:
            print(f"Timed out waiting for {xpath} to become interactable.")

    def publish(self):
        if self.retry_count < 3:  # maximum 3 tries (initial + 2 retries)
            try:
                driver = self.driver
                path_mp4 = self.path_mp4
                path_cover = self.path_cover
                metadata = self.metadata
                test = self.test

                print("Starting the publishing process on XiaoHongShu...")
                driver.get("https://creator.xiaohongshu.com/creator/post")
                print("Navigated to post creation page.")

                time.sleep(1)
                dismiss_alert(driver)
                time.sleep(10)

                bring_to_front(["小红书", "你访问的页面不见了"])

                WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//input[@type="file"]')))
                print("Video upload field is present.")

                print(f"Uploading video from path: {path_mp4}")
                time.sleep(3)
                driver.find_element(By.XPATH, '//input[@type="file"]').send_keys(path_mp4)

                # Monitor upload status
                reupload_xpath = '//*[contains(text(),"替换视频")]'  # or '//*[contains(text(),"重新上传")]'
                failure_xpath = '//*[contains(text(),"上传失败")]'
                time.sleep(3)
                start_time = time.time()
                timeout = 3600  # 3600 seconds timeout

                while True:
                    if time.time() - start_time > timeout:
                        raise Exception("Timeout reached while waiting for video to be uploaded or for a failure message.")

                    try:
                        # Wait until the '替换视频' (or '重新上传') element is present
                        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, reupload_xpath)))
                        print("Video uploaded successfully!")
                        break
                    except:
                        pass

                    try:
                        # Wait until the "上传失败" element is present
                        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, failure_xpath)))
                        print("Upload failed! Raising an error to initiate retry...")
                        raise Exception("Video upload failed.")
                    except:
                        pass

                    time.sleep(5)

                print("Entering title and description.")
                time.sleep(3)
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//*[contains(@class,"titleInput")]//input'))
                )
                driver.find_element(By.XPATH, '//*[contains(@class,"titleInput")]//input').send_keys(metadata['title'][:20])

                description_with_tags = metadata['long_description'] + " " + " ".join([f"#{tag}" for tag in metadata['tags']])
                time.sleep(3)
                # driver.find_element(By.XPATH, '//*[contains(@class,"topic-container")]//p').send_keys(description_with_tags[:1000])
                
                driver.find_element(
                    By.XPATH,
                    '//div[contains(@class,"ql-editor") and @contenteditable="true"]'
                ).send_keys(description_with_tags[:1000])

                
#                 try:
#                     print("Waiting for the '修改封面' button to become clickable...")
#                     cover_button_xpath = "//button[not(@disabled) and contains(., '修改封面')]"
#                     WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, cover_button_xpath)))

#                     print("Clicking on the '修改封面' button...")
#                     driver.find_element(By.XPATH, cover_button_xpath).click()
#                     print("Button clicked successfully! Proceeding with further actions.")

#                     time.sleep(3)
#                     print("Waiting 上传封面...")
#                     self.wait_for_element_to_be_clickable('//*[text()="上传封面"]')
#                     driver.find_element(By.XPATH, '//*[text()="上传封面"]').click()

#                     time.sleep(3)
#                     print(f"Uploading cover from path: {path_cover}")
#                     file_input_xpath = '//input[@class="upload-input"]'
#                     time.sleep(3)
#                     driver.find_element(By.XPATH, file_input_xpath).send_keys(path_cover)
#                     time.sleep(3)

#                     print("Waiting for the '确定' button to be clickable...")
#                     confirm_button_xpath = "//button[contains(@class, 'btn-confirm') and contains(., '确定')]"
#                     WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.XPATH, confirm_button_xpath)))

#                     print("Clicking on the '确定' button...")
#                     driver.find_element(By.XPATH, confirm_button_xpath).click()
#                     print("Button clicked successfully! Proceeding with further actions.")

#                     cover_uploaded_button_xpath = '//*[text()="修改默认封面"]'
#                     time.sleep(3)
#                     WebDriverWait(driver, 600).until(EC.presence_of_element_located((By.XPATH, cover_uploaded_button_xpath)))
#                     print("Cover uploaded successfully! Proceeding to location selection.")

#                 except Exception as e:
#                     print("Cover upload with error: ", str(e))

#                 def select_location(driver, location_names, retry_count=2):
#                     try:
#                         location_name = location_names[0]
#                         print(f"Attempting to select location: {location_name}")
#                         time.sleep(3)

#                         input_box = WebDriverWait(driver, 10).until(
#                             EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='请选择地点']"))
#                         )
#                         print("Location input box is clickable.")
#                         input_box.click()
#                         input_box.send_keys(location_name)
#                         input_box.send_keys(Keys.RETURN)
#                         print(f"Entered location: {location_name}")

#                         time.sleep(3)
#                         WebDriverWait(driver, 10).until(
#                             EC.visibility_of_element_located((By.CLASS_NAME, "el-autocomplete-suggestion"))
#                         )
#                         print("Dropdown suggestions are visible.")

#                         time.sleep(3)
#                         print(f"Executing JavaScript to click on the location option '{location_name}'")
#                         js_code = f"""
#                         var options = document.querySelectorAll(".el-autocomplete-suggestion__list .item .name");
#                         options.forEach(function(option) {{
#                             if (option.innerText === "香港大学") {{
#                                 option.click();
#                                 console.log("Selected option: '{location_name}'");
#                             }}
#                         }});
#                         """
#                         driver.execute_script(js_code)
#                         print(f"Location '{location_name}' selected successfully!")
#                     except TimeoutException as te:
#                         print(f"TimeoutException: {te}")
#                         if retry_count > 0 and len(location_names) > 1:
#                             print(f"Retrying to select location... Attempts left: {retry_count - 1}")
#                             select_location(driver, location_names[1:], retry_count - 1)
#                         else:
#                             print("Failed to select location after multiple attempts.")
#                     except Exception as e:
#                         print(f"Exception: {e}")
#                         traceback.print_exc()

#                 # For the first publish attempt only, we try selecting location
#                 if self.retry_count == 0:
#                     select_location(driver, ['The University', '香港大学', 'The University of Hong Kong'])

                # 1) Click the “添加地点” dropdown
                dropdown_placeholder = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable(
                        (By.XPATH, '//div[contains(@class,"d-select-placeholder") and text()="添加地点"]')
                    )
                )
                dropdown_placeholder.click()

                # 2) Locate the actual text input (be sure to match the correct input box)
                address_input = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located(
                        (By.XPATH, '//div[@class="d-select-input-filter"]/input[@class="d-text"]')
                    )
                )

                # 3) Type "The University" only
                address_input.send_keys("香港大学")
                time.sleep(2)  # Let suggestions populate



                # Prompt the user to confirm publishing
                if test:
                    user_input = input("Do you want to publish now? Type 'yes' to confirm: ").strip().lower()
                else:
                    user_input = "yes"

                if user_input == 'yes':
                    time.sleep(3)
                    publish_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[text()="发布"]')))
                    time.sleep(3)
                    publish_button.click()
                    time.sleep(10)
                    dismiss_alert(driver)
                    time.sleep(3)
                    print("Publishing...")
                else:
                    print("Publishing cancelled by user.")

                print("Process completed successfully!")
                self.retry_count = 0  # reset after success

            except Exception as e:
                print(f"An error occurred: {e}")
                traceback.print_exc()
                self.retry_count += 1
                print(f"Retrying the whole process... Attempt {self.retry_count}")
                self.publish()
        else:
            print("Maximum retry attempts reached. Process failed.")

In [4]:
# ---------------------------------------------------------------------
# Example of usage in Jupyter
# You can place everything in a single cell, then run these lines:
driver = create_new_driver(port=5003)  # Adjust port to match your debugger setup

# Explicit video and cover paths
video_path = "/home/lachlan/Projects/auto-publish/transcription_data/IMG_5197_2024_09_22_21_01_17_COMPLETED/IMG_5197_2024_09_22_21_01_17_COMPLETED_highlighted.mp4"
cover_path = "/home/lachlan/Projects/auto-publish/transcription_data/IMG_5197_2024_09_22_21_01_17_COMPLETED/IMG_5197_2024_09_22_21_01_17_COMPLETED_cover.jpg"
metadata_path = "/home/lachlan/Projects/auto-publish/transcription_data/IMG_5197_2024_09_22_21_01_17_COMPLETED/IMG_5197_2024_09_22_21_01_17_COMPLETED_metadata.json"

# Load metadata from JSON
with open(metadata_path, "r") as f:
    metadata = json.load(f)

# Create our streamlined publisher instance
publisher_xhs = XiaoHongShuPublisher(
    driver=driver,
    path_mp4=video_path,
    path_cover=cover_path,
    metadata=metadata,
    test=True  # Set True so you have a chance to confirm or cancel
)

publisher_xhs.publish()

Checking if XHS is logged in. (Stub)
Starting the publishing process on XiaoHongShu...
Navigated to post creation page.
Focusing on ['小红书', '你访问的页面不见了'] if found. (Stub)
Video upload field is present.
Uploading video from path: /home/lachlan/Projects/auto-publish/transcription_data/IMG_5197_2024_09_22_21_01_17_COMPLETED/IMG_5197_2024_09_22_21_01_17_COMPLETED_highlighted.mp4
Video uploaded successfully!
Entering title and description.
An error occurred: Message: 
Stacktrace:
#0 0x5556160a75f4 <unknown>
#1 0x555615d97264 <unknown>
#2 0x555615ddd77c <unknown>
#3 0x555615e1cf20 <unknown>
#4 0x555615dd0a2c <unknown>
#5 0x555615dd1c44 <unknown>
#6 0x55561607160c <unknown>
#7 0x5556160749c0 <unknown>
#8 0x555616061aec <unknown>
#9 0x55561607537c <unknown>
#10 0x555616049ea0 <unknown>
#11 0x5556160953f0 <unknown>
#12 0x5556160955bc <unknown>
#13 0x5556160a67a8 <unknown>
#14 0x7ffee11cee18 <unknown>
#15 0x7ffee1237e9c <unknown>

Retrying the whole process... Attempt 1
Starting the publishing pr

Traceback (most recent call last):
  File "/tmp/ipykernel_10344/103657020.py", line 253, in publish
    address_input = WebDriverWait(driver, 10).until(
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/lachlan/miniconda3/envs/autopub/lib/python3.12/site-packages/selenium/webdriver/support/wait.py", line 105, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
#0 0x5556160a75f4 <unknown>
#1 0x555615d97264 <unknown>
#2 0x555615ddd77c <unknown>
#3 0x555615e1cf20 <unknown>
#4 0x555615dd0a2c <unknown>
#5 0x555615dd1c44 <unknown>
#6 0x55561607160c <unknown>
#7 0x5556160749c0 <unknown>
#8 0x555616061aec <unknown>
#9 0x55561607537c <unknown>
#10 0x555616049ea0 <unknown>
#11 0x5556160953f0 <unknown>
#12 0x5556160955bc <unknown>
#13 0x5556160a67a8 <unknown>
#14 0x7ffee11cee18 <unknown>
#15 0x7ffee1237e9c <unknown>



Navigated to post creation page.
Focusing on ['小红书', '你访问的页面不见了'] if found. (Stub)
Video upload field is present.
Uploading video from path: /home/lachlan/Projects/auto-publish/transcription_data/IMG_5197_2024_09_22_21_01_17_COMPLETED/IMG_5197_2024_09_22_21_01_17_COMPLETED_highlighted.mp4


KeyboardInterrupt: 